<a href="https://colab.research.google.com/github/mohammadp1001/RenewableInsight/blob/main/playground_s5e11_fe_20260218_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

No categorical features.

In [1]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
# Works in Kaggle + Colab
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:

import os
import json
import time
import ydf
import numpy as np
import pandas as pd
import xgboost as xgb

from dataclasses import dataclass, asdict

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

import wandb

from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [4]:
def clean_col_names(df):
    cols = df.columns
    new_cols = []
    for col in cols:
        new_col = col.lower().replace(' ', '_')
        new_cols.append(new_col)
    df.columns = new_cols
    return df

In [5]:
playground_series_s6e2_path = kagglehub.competition_download('playground-series-s6e2')
# original_dataset_s6e2_path = kagglehub.dataset_download('neurocipher/heartdisease')
print('Data source import complete.')

train_file_path = os.path.join(playground_series_s6e2_path, 'train.csv')
test_file_path = os.path.join(playground_series_s6e2_path, 'test.csv')
# original_dataset_s6e2_path = os.path.join(original_dataset_s6e2_path, 'Heart_Disease_Prediction.csv')
df_train = pd.read_csv(train_file_path)
df_test = pd.read_csv(test_file_path)
# df_original = pd.read_csv(original_dataset_s6e2_path)
print(f"\nSuccessfully loaded 'train.csv' into df_train:")
print(f"Shape of df_train: {df_train.shape}")

df_train = clean_col_names(df_train)
df_test = clean_col_names(df_test)
# df_original = clean_col_names(df_original)


100%|██████████| 10.2M/10.2M [00:00<00:00, 159MB/s]

Extracting files...


Data source import complete.

Successfully loaded 'train.csv' into df_train:
Shape of df_train: (630000, 15)


In [6]:
X_test = df_test.drop(columns=['id'], axis=1)
label_mapping = {'Absence': 0, 'Presence': 1}
df_train['heart_disease'] = df_train['heart_disease'].map(label_mapping)
X = df_train.drop(columns=['heart_disease','id'], axis=1)
y = df_train['heart_disease']

In [7]:
@dataclass
class CVConfig:
    project: str = "playground_s5e11"
    entity: str | None = None
    run_name: str | None = None
    group: str | None = None
    tags: tuple = ("cv", "oof")
    n_splits: int = 5
    seed: int = 42
    save_dir: str = "/content/models"


def run_cv_oof_binary(
    X,
    y,
    X_test,
    cfg: CVConfig,
    fit_fold_fn,       # (X_tr, y_tr, X_va, y_va, fold, cfg, run) -> (model, val_proba_pos)
    model_params: dict,
):
    os.makedirs(cfg.save_dir, exist_ok=True)
    y = np.asarray(y).astype(int)


    oof = np.zeros((len(y),), dtype=np.float32)
    test_prop_pos = np.zeros((len(X_test),), dtype=np.float32)
    skf = StratifiedKFold(n_splits=cfg.n_splits, shuffle=True, random_state=cfg.seed)

    run = wandb.init(
        project=cfg.project,
        entity=cfg.entity,
        name=cfg.run_name,
        group=cfg.group,
        tags=list(cfg.tags),
        config={
            **asdict(cfg),
            "model_params": model_params,
            "n_samples": int(X.shape[0]),
            "n_features": int(X.shape[1]),
            "metric": "auc",
        },
    )

    fold_scores = []
    t0 = time.time()

    try:
        for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), start=1):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
            y_tr, y_va = y[tr_idx], y[va_idx]

            model, val_proba_pos, test_prop_pos = fit_fold_fn(X_tr, y_tr, X_va, y_va, X_test, fold, cfg, run)

            oof[va_idx] = val_proba_pos
            fold_auc = roc_auc_score(y_va, val_proba_pos)
            fold_scores.append(fold_auc)

            run.log({"fold": fold, "fold_auc": fold_auc})
            test_prop_pos += test_prop_pos

        oof_auc = roc_auc_score(y, oof)
        run.summary["oof_auc"] = float(oof_auc)
        run.summary["fold_auc_mean"] = float(np.mean(fold_scores))
        run.summary["fold_auc_std"] = float(np.std(fold_scores))
        run.summary["elapsed_sec"] = float(time.time() - t0)

        oof_path = os.path.join(cfg.save_dir, "oof_pos.npy")
        np.save(oof_path, oof)
        run.save(oof_path)

        print(f"OOF AUC: {oof_auc:.6f}")
        return oof, test_prop_pos/cfg.n_splits

    finally:
        wandb.finish()


# -----------------------------
# Fold fit functions (binary)
# -----------------------------
def fit_fold_xgb_binary(X_tr, y_tr, X_va, y_va, X_test, fold, cfg, run):

    params = run.config["model_params"]
    dtrain = xgb.DMatrix(X_tr, label=y_tr,enable_categorical=True)
    dval = xgb.DMatrix(X_va, label=y_va,enable_categorical=True)
    dtest = xgb.DMatrix(X_test,enable_categorical=True)

    with xgb.config_context(use_rmm=True):
      model = xgb.train(
          params=params,
          dtrain=dtrain,
          evals=[(dtrain, "train"),
                (dval, "valid")],
      )


    path = os.path.join(cfg.save_dir, f"xgb_fold{fold}.json")
    model.save_model(path)
    run.save(path)

    val_proba_pos = model.predict(dval)
    test_prop_pos = model.predict(dtest)

    return model, val_proba_pos, test_prop_pos


def fit_fold_lgbm_binary(X_tr, y_tr, X_va, y_va,X_test, fold, cfg, run):

    params = run.config["model_params"]
    model = LGBMClassifier(**params)

    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        callbacks=[
            lgb.early_stopping(50, first_metric_only=True),
            lgb.log_evaluation(100),
        ],
    )

    path = os.path.join(cfg.save_dir, f"lgbm_fold{fold}.txt")
    model.booster_.save_model(path)
    run.save(path)

    val_proba_pos = model.predict_proba(X_va)[:, 1]
    test_prop_pos = model.predict_proba(X_test)[:, 1]
    return model, val_proba_pos, test_prop_pos


def fit_fold_catboost_binary(X_tr, y_tr, X_va, y_va, X_test, fold, cfg, run, cat_features=None):


    params = run.config["model_params"]

    train_pool = Pool(X_tr, y_tr, cat_features=cat_features)
    val_pool = Pool(X_va, y_va, cat_features=cat_features)

    model = CatBoostClassifier(**params)
    model.fit(train_pool, eval_set=val_pool, verbose=100)

    path = os.path.join(cfg.save_dir, f"catboost_fold{fold}.cbm")
    model.save_model(path)
    run.save(path)

    val_proba_pos = model.predict_proba(X_va)[:, 1]
    test_prop_pos = model.predict_proba(X_test)[:, 1]
    return model, val_proba_pos, test_prop_pos


In [8]:
os.environ['WANDB_API_KEY'] = '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'

In [9]:
cfg = CVConfig(run_name="xgb_20260218_3", group="xgb")
xgb_params = dict(
     objective="binary:logistic",
     tree_method="hist",
     device="cuda",
     learning_rate=0.01,
     max_depth=8,
     subsample=0.86,
     colsample_bytree=0.467,
     gamma=0.26,
     reg_alpha=2.7,
     reg_lambda=1.4,
     random_state=120,
     eval_metric="auc",              # internal metric; still compute OOF AUC
)
oof_xgb, test_prop_pos_xgb = run_cv_oof_binary(X, y, X_test, cfg, fit_fold_xgb_binary, xgb_params)


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from WANDB_API_KEY.
wandb: Currently logged in as: mohammadp1001 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[0]	train-auc:0.89947	valid-auc:0.89972
[1]	train-auc:0.91194	valid-auc:0.91209
[2]	train-auc:0.93919	valid-auc:0.93933
[3]	train-auc:0.94080	valid-auc:0.94082
[4]	train-auc:0.95003	valid-auc:0.95018
[5]	train-auc:0.95141	valid-auc:0.95150
[6]	train-auc:0.95246	valid-auc:0.95246
[7]	train-auc:0.95246	valid-auc:0.95250
[8]	train-auc:0.95276	valid-auc:0.95285
[9]	train-auc:0.95202	valid-auc:0.95205


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")
wandb: WARNING Symlinked 1 file into the W&B run directory; call wandb.save again to sync new files.


[0]	train-auc:0.90007	valid-auc:0.89799
[1]	train-auc:0.91226	valid-auc:0.91044
[2]	train-auc:0.93955	valid-auc:0.93775
[3]	train-auc:0.94123	valid-auc:0.93922
[4]	train-auc:0.95034	valid-auc:0.94881
[5]	train-auc:0.95168	valid-auc:0.95004
[6]	train-auc:0.95271	valid-auc:0.95126
[7]	train-auc:0.95271	valid-auc:0.95126
[8]	train-auc:0.95301	valid-auc:0.95157
[9]	train-auc:0.95229	valid-auc:0.95062


wandb: WARNING Symlinked 1 file into the W&B run directory; call wandb.save again to sync new files.


...

wandb: WARNING Symlinked 1 file into the W&B run directory; call wandb.save again to sync new files.


[0]	train-auc:0.89955	valid-auc:0.90041
[1]	train-auc:0.91165	valid-auc:0.91280
[2]	train-auc:0.93911	valid-auc:0.93996
[3]	train-auc:0.94067	valid-auc:0.94158
[4]	train-auc:0.94996	valid-auc:0.95047
[5]	train-auc:0.95132	valid-auc:0.95175
[6]	train-auc:0.95236	valid-auc:0.95279
[7]	train-auc:0.95235	valid-auc:0.95270
[8]	train-auc:0.95264	valid-auc:0.95288
[9]	train-auc:0.95189	valid-auc:0.95218


wandb: WARNING Symlinked 1 file into the W&B run directory; call wandb.save again to sync new files.
wandb: WARNING Symlinked 1 file into the W&B run directory; call wandb.save again to sync new files.


OOF AUC: 0.951592


fold,▁▃▅▆█
fold_auc,▇▁▇▃█
elapsed_sec,3.4936
fold,5
fold_auc,0.95218
fold_auc_mean,0.9516
fold_auc_std,0.00061
oof_auc,0.95159
